**This jupyther notebook desing and implement** a **meditron finetuning** inspered on project source code from [Meditron in github repository](https://github.com/epfLLM/meditron) and some other resouces as [
    Finetune LLMs on your own consumer hardware using tools from PyTorch and Hugging Face ecosystem](https://pytorch.org/blog/finetune-llms/)

El método de fine-tuning se basará en un método reciente llamado "Low Rank Adapters" (LoRA), en lugar de ajustar todo el modelo sólo tienes que ajustar estos adaptadores y cargarlos correctamente dentro del modelo. Después de afinar el modelo también puedes compartir tus adaptadores en el Hub 🤗 y cargarlos muy fácilmente.

In [1]:
! nvidia-smi

Wed Mar 13 15:28:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Load dependencies


In [10]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Hugginface loggin for push on Hub

In [4]:
from huggingface_hub import notebook_login
notebook_login()

[link text](https://)###Load the model

Aquí vamos a cargar el modelo bertin-project/bertin-gpt-j-6B, ¡sus pesos en media precisión (float16) ocupan unos 13GB en el Hub! Si los cargamos en 8 bits necesitaríamos unos 7GB de memoria.

In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

model_id = 'epfl-llm/meditron-7b'

In [18]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True,
                                         llm_int8_threshold=200.0)

In [19]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto',
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Avoid a warning wiht the model**

In [20]:
tokenizer.pad_token = tokenizer.eos_token

In [21]:
new_tokens = ["<SC>","<EC>"]

num_added_toks = tokenizer.add_tokens(new_tokens)
print("We have added", num_added_toks, "tokens")
# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
model.resize_token_embeddings(len(tokenizer))

We have added 2 tokens


Embedding(32019, 4096)

Por último, tenemos que aplicar algunos ajustes de post-procesamiento en el modelo de 8 bits para que se pueda entrenar correctamente, vamos a congelar todas nuestras capas, y el hacer cast de la capa de normalización en float32 por cuestiones de estabiliad. Por las mismas razones hacemos lo mismo con la última capa.

In [22]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads() #Permit an efficient way

class CastOutputToFloat(nn.Sequential): #Cast the last layer
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Aplicar LoRA

¡Aquí viene la magia con `peft`! Vamos a cargar un `PeftModel` y especificar que vamos a utilizar adaptadores de bajo rango (LoRA) utilizando `get_peft_model` función de utilidad de `peft`.

In [23]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

parameter 'r': Count of independe variable in each Lora Matrix, in other paper used 8, but r=8 and lora_alpha=16 not affet the result. For more information about that read the fundational paper Lora.

parameter 'lora_alpha':

parameter 'target_modules': module of base model where will be apply the Lora action.

parameter 'lora_dropout': Classical droput for action event.


In [24]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8388608 || all params: 6746959872 || trainable%: 0.12433167173281803


### Carga y procesado del dataset

In [25]:
import transformers
from datasets import load_dataset

In [26]:
dataset_id = 'epfl-llm/guidelines'

In [56]:
dataset = load_dataset(dataset_id)

In [57]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'title', 'clean_text', 'raw_text', 'url', 'overview'],
        num_rows: 37970
    })
})

Modified the text parameter on instance in dataset add the token start and token end

In [64]:
dataset = dataset['train'].train_test_split(test_size=0.3)

In [59]:
def format_ds(example):
  example["text"] = "<SC>" + example['clean_text'] + "<EC>"
  return example

In [65]:
dataset_test = dataset['test'].map(format_ds, remove_columns=['id', 'title', 'raw_text', 'source','url','overview'])

Map:   0%|          | 0/11391 [00:00<?, ? examples/s]

In [66]:
dataset_train = dataset['train'].map(format_ds, remove_columns=['id', 'title', 'raw_text', 'source','url','overview'])

Map:   0%|          | 0/26579 [00:00<?, ? examples/s]

In [67]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'title', 'clean_text', 'raw_text', 'url', 'overview'],
        num_rows: 26579
    })
    test: Dataset({
        features: ['id', 'source', 'title', 'clean_text', 'raw_text', 'url', 'overview'],
        num_rows: 11391
    })
})

In [68]:
dataset_train[0]

{'clean_text': 'Chromosome 1, 1p36 deletion syndrome\nChromosome 1, 1p36 deletion syndrome is a medical condition caused by a rare chromosomal disorder where deletion of a portion of chromosome 1 causes various abnormalities such as heart defects, mental retardation, developmental delay, facial dysmorphism and short stature. The symptoms are variable depending on the exact location of chromosomal deletion.\n\n# History\nThe first cases of 1p36 deletion syndrome were described in the 1980s. However, since many of these individuals also had other chromosomal imbalances, symptoms varied widely. The reason it took so long to recognize the condition as a distinct chromosome deletion syndrome is that the deletions causing the disorder are too small to be detected in a routine chromosomal analysis. FISH (fluorescent in situ hybridization) and DNA-based technology known as MPLA (multiple ligation probe amplification) used in testing have aided in diagnosing an increasing number of cases since 

**Split dataset to train**

In [69]:
dataset_train = dataset_train.map(lambda samples: tokenizer(samples['clean_text']), batched=True)

Map:   0%|          | 0/26579 [00:00<?, ? examples/s]

In [70]:
dataset_train

Dataset({
    features: ['clean_text', 'text', 'input_ids', 'attention_mask'],
    num_rows: 26579
})

*fp16=True* For preserve memory

### Configuración de los parámetros de entrenamiento

In [72]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=8,
        warmup_steps=100,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=5,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)